In [17]:
import sys
sys.path.append("/home/pervinco/Upstage_Ai_Lab/Final/IR/src")

import os
import copy
import json

from tqdm import tqdm
from openai import OpenAI
from langchain.schema import Document

from dotenv import load_dotenv
load_dotenv("../keys.env")

openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ['OPENAI_API_KEY'] = openai_api_key

from data.data import load_document, load_query

In [18]:
def load_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

def load_document(path='../dataset/documents.jsonl'):
    raw_documents = load_jsonl(path)

    documents = []
    for doc in raw_documents:
        docid = doc['docid']
        src = doc['src']
        content = doc['content']
        documents.append(Document(page_content=content, metadata={"docid": docid, "src": src}))

    return documents

In [5]:
document_path = "../dataset/processed_documents.jsonl"
documents = load_document(document_path)

print(len(documents))

4272


In [6]:
arc_list = []
mmlu_list = []
for doc in documents:
    docid = doc.metadata['docid']
    src = doc.metadata['src']

    src_splited = src.split("__")
    if src_splited[0] == "ko_mmlu":
        mmlu_list.append({'docid':docid, 'content':doc.page_content, 'domain':src_splited[1]})
    else:
        arc_list.append((docid, doc.page_content))

print(len(arc_list))
print(len(mmlu_list))

2047
2225


In [7]:
mmlu_domain_set = set([x['domain'] for x in mmlu_list])
print(len(mmlu_domain_set))
print(mmlu_domain_set)

20
{'computer_security', 'high_school_biology', 'college_computer_science', 'virology', 'college_physics', 'human_aging', 'global_facts', 'conceptual_physics', 'college_medicine', 'nutrition', 'anatomy', 'astronomy', 'college_chemistry', 'high_school_physics', 'high_school_computer_science', 'electrical_engineering', 'high_school_chemistry', 'medical_genetics', 'college_biology', 'human_sexuality'}


In [8]:
total_set = {'high_school_biology', 'microbiology', 'medical_genetics', 'agriculture', 'college_biology',
             'technology', 'plant_biology', 'college_medicine', 'neuroscience', 'evolution',
             'environmental_science', 'materials_science', 'global_facts', 'engineering', 'meteorology',
             'college_chemistry', 'biology', 'high_school_computer_science', 'physics', 'aviation',
             'chemistry', 'education', 'biotechnology', 'human_sexuality', 'computer_security',
             'high_school_physics', 'music', 'safety', 'ecology', 'data_visualization',
             'entomology', 'optics', 'marketing', 'college_computer_science', 'geology',
             'animal_behavior', 'energy', 'genetics', 'nuclear_physics', 'biochemistry',
             'anatomy', 'acoustics', 'nutrition', 'immunology', 'scientific_method',
             'virology', 'cell_biology', 'logistics', 'college_physics', 'astronomy',
             'human_aging', 'sports', 'paleontology', 'oceanography', 'electrical_engineering',
             'conceptual_physics', 'high_school_chemistry'}

In [9]:
client = OpenAI()
model = "gpt-4o"

def generate_domain(inps, model, client):
    context = (
    """
    먼저, 주어진 문서 내용을 보고 핵심 내용을 파악해주세요.
    그런 다음 함께 제공된 리스트는 사전 정의된 도메인 집합의 원소들 중 가장 적합한 도메인 하나를 선택해서 반환해주세요.

    반환하는 형태는 해당 문서가 속하는 도메인 영어 단어 하나입니다.
    단어에 따옴표(')나 쌍따옴표(")를 붙이지 않아야 합니다.
    """
    )
    
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": context},
            {"role": "user", "content": inps}
        ],
    )
    
    response = completion.choices[0].message.content
    
    return response

In [10]:
arc_domain_list = []
pre_domain_set = copy.copy(mmlu_domain_set)


for idx, arc in enumerate(arc_list):
    docid, page_content = arc[0], arc[1]
    print(idx)
    print(docid)
    print(page_content)

    domain = generate_domain(f"{page_content}\n{total_set}", model, client)
    print(f"{domain}\n")
    
    arc_domain_list.append({'docid':docid, 'content':page_content, 'domain':domain})
    pre_domain_set.add(domain)

0
d3c68be5-9cb1-4d6e-ba18-5f81cf89affb
종이와 플라스틱은 재활용 가능한 자원입니다. 중학교 과학 수업에서 우리는 종이와 플라스틱 가방을 재활용하는 프로젝트를 진행하고 있습니다. 이 프로젝트를 통해 우리는 환경 보호의 중요성을 배우고, 지구를 지키기 위해 우리가 할 수 있는 작은 노력을 실천하고 있습니다. 종이와 플라스틱의 주요 차이점은 종이가 재생 가능한 자원에서 나온다는 것입니다. 종이는 나무에서 추출되는 천연 자원으로, 나무를 재배하고 재생하는 과정을 통해 생산됩니다. 반면에 플라스틱은 석유에서 추출되는 인공적인 자원으로, 석유의 가공 과정을 통해 생산됩니다. 이러한 차이로 인해 종이는 자연적으로 분해되는 속성을 가지고 있어서 자연 환경에 미치는 영향이 적습니다. 그러나 플라스틱은 자연 분해가 어렵기 때문에 환경 오염의 주요 원인 중 하나입니다. 따라서 우리는 종이와 플라스틱을 재활용하여 자원을 보존하고 환경 오염을 줄이는 데에 기여할 수 있습니다. 이 프로젝트를 통해 우리는 지구를 더욱 건강하고 지속 가능한 곳으로 만들기 위해 노력하고 있습니다.
environmental_science

1
910107a6-2a42-41a2-b337-fbf22d6440fe
마이애미파랑나비는 남부 플로리다에서 멸종 위기에 처한 종입니다. 이 나비의 개체수 감소를 초래했을 가능성이 가장 높은 요인은 주택 건설 증가입니다. 남부 플로리다 지역에서는 최근 몇 년간 주택 건설이 급증하였고, 이로 인해 서식지가 감소하고 개체들의 서식 환경이 파괴되었습니다. 마이애미파랑나비는 특정한 식물에 의존하여 번식하고 생존하는데, 주택 건설로 인해 이 식물들의 서식지가 감소하였기 때문에 개체수가 감소한 것으로 추정됩니다. 이러한 주택 건설 증가는 마이애미파랑나비뿐만 아니라 다른 생물종들에게도 영향을 미치고 있으며, 생태계의 균형을 위해 적절한 조치가 필요합니다. 주택 건설 증가를 제한하거나 대안적인 서식지를 마련하는 등의 방안이 고려되어야 합니다.
environme

In [11]:
arc_domain_set = set([x['domain'] for x in  arc_domain_list])
total_set = mmlu_domain_set.union(arc_domain_set)

print(len(total_set))
print(total_set)

62
{'high_school_biology', 'cell_biology', 'virology', 'music', 'global_facts', 'conceptual_physics', 'immunology', 'data_visualization', 'aviation', 'agriculture', 'geology', 'nutrition', 'anatomy', 'high_school_physics', 'high_school_computer_science', 'electrical_engineering', 'high_school_chemistry', 'scientific_method', 'college_biology', 'microbiology', 'physics', 'human_sexuality', 'environmental_science', 'computer_security', 'neuroscience', 'sports', 'college_computer_science', 'oceanography', 'engineering', 'nuclear_physics', 'paleontology', 'safety', 'marketing', 'evolution', 'optics', 'college_physics', 'education', 'plant_biology', 'biochemistry', 'epigenetics', 'human_aging', 'biotechnology', 'endocrinology', 'chemistry', 'meteorology', 'technology', 'college_medicine', 'ecology', 'energy', 'acoustics', 'astronomy', 'materials_science', 'genetics', 'food_preparation', 'college_chemistry', 'biology', 'aquaculture', "pollinator-plant interaction과 관련된 내용이므로 해당 문서가 속하는 도메인은 '

In [12]:
merged_domain_list = mmlu_list + arc_domain_list
output_file = "../dataset/processed_domain_documents.jsonl"

with open(output_file, 'w', encoding='utf-8') as f:
    for item in merged_domain_list:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")


In [13]:
print(merged_domain_list[0])

{'docid': '42508ee0-c543-4338-878e-d98c6babee66', 'content': '건강한 사람이 에너지 균형을 평형 상태로 유지하는 것은 중요합니다. 에너지 균형은 에너지 섭취와 에너지 소비의 수학적 동등성을 의미합니다. 일반적으로 건강한 사람은 1-2주의 기간 동안 에너지 균형을 달성합니다. 이 기간 동안에는 올바른 식단과 적절한 운동을 통해 에너지 섭취와 에너지 소비를 조절해야 합니다. 식단은 영양가 있는 식품을 포함하고, 적절한 칼로리를 섭취해야 합니다. 또한, 운동은 에너지 소비를 촉진시키고 근육을 강화시킵니다. 이렇게 에너지 균형을 유지하면 건강을 유지하고 비만이나 영양 실조와 같은 문제를 예방할 수 있습니다. 따라서 건강한 사람은 에너지 균형을 평형 상태로 유지하는 것이 중요하며, 이를 위해 1-2주의 기간 동안 식단과 운동을 조절해야 합니다.', 'domain': 'nutrition'}


In [14]:
docid_domain_map = {}

for doc in merged_domain_list:
    docid_domain_map[doc['docid']] = doc['domain']

print(len(docid_domain_map))

4272


In [15]:
def load_jsonl(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [json.loads(line) for line in f]

def load_train_document(path):
    raw_documents = load_jsonl(path)

    documents = []
    for doc in raw_documents:
        docid = doc['docid']
        content = doc['content']
        documents.append(Document(page_content=content, metadata={"docid": docid}))

    return documents

train_documents = load_train_document("../dataset/gpt_contextual_retrieval_documents_v3.jsonl")

domain_added = []
for doc in train_documents:
    train_doc_id = doc.metadata['docid']
    train_doc_content = doc.page_content

    domain = docid_domain_map[train_doc_id]
    domain_added.append({"docid":train_doc_id, 
                         "domain":domain,
                         "content":train_doc_content})

In [16]:
with open("../dataset/gpt_contextual_retrieval_documents_v5.jsonl", 'w', encoding='utf-8') as f:
    for item in merged_domain_list:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")

In [13]:
client = OpenAI()
model = "gpt-4o"

def generate_domain(inps, model, client):
    context = (
    """
    주어진 질의 내용을 보고 해당 질의가 어떤 분야에 대한 것인지 함께 입력한 사전 정의된 도메인 리스트 중 하나로 선택해 반환해주세요.

    반환하는 형태는 해당 질의가 속하는 도메인 영어 단어 하나입니다.
    단어에 따옴표(')나 쌍따옴표(")를 붙이지 않아야 합니다.
    """
    )
    
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": context},
            {"role": "user", "content": inps}
        ],
    )
    
    response = completion.choices[0].message.content
    
    return response

In [ ]:
queries = load_jsonl("../dataset/processed_eval.jsonl")

eval_domain_set = copy.copy(total_set)
domain_eval = []
for query in queries:
    eval_id = query['eval_id']
    question = query['query']['query']
    print(eval_id)
    print(question)

    domain = generate_domain(f"{question}\n{total_set}", model, client)
    print(domain)
    print()

    eval_domain_set.add(domain)
    domain_eval.append({"eval_id":eval_id, "query":question, "domain":domain})

In [ ]:
total_set.difference(eval_domain_set)

In [16]:
with open("../dataset/domain_eval.jsonl", 'w', encoding='utf-8') as f:
    for item in domain_eval:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")

In [ ]:
print(len(total_set))
print(total_set)

In [19]:
queries = load_jsonl("../dataset/processed_eval.jsonl")

eval_domain_set = copy.copy(total_set)
domain_eval = []
for query in queries:
    eval_id = query['eval_id']
    question = query['query']['query']
    print(eval_id)
    print(question)

    domain_eval.append({"eval_id":eval_id, "query":question})

with open("../dataset/processed_eval.jsonl", 'w', encoding='utf-8') as f:
    for item in domain_eval:
        json.dump(item, f, ensure_ascii=False)
        f.write("\n")

78
나무의 분류에 대해 조사해 보기 위한 방법은?
213
각 나라에서의 공교육 지출 현황에 대해 알려줘.
107
기억 상실증의 원인은 무엇인가요?
81
통학 버스의 가치에 대해 말해줘.
280
Dmitri Ivanovsky가 누구야?
10
피임을 하기 위한 방법중 약으로 처리하는 방법은 쓸만한가?
100
헬륨이 다른 원소들과 반응을 잘 안하는 이유는?
279
문맹 비율이 사회 발전에 미치는 영향은?
42
이란 콘트라 사건이 무엇인지와 이 사건이 미국 정치에 미친 영향을 설명해 주세요.
308
자기장이 얼마나 센지 표현하는 방식은?
205
피를 맑게 하고 몸 속의 노폐물을 없애는 역할을 하는 기관은?
289
글리코겐의 분해는 인체에서 왜 필요한가?
268
빗방울이 점점 커지게 되는 요인은?
18
기체의 부피나 형태가 왜 일정하지 않을까?
9
식물이 빛을 에너지로 변환하는 과정에 대해 설명해줘.
101
직류와 교류 전류의 차이에 대해 알려줘.
236
기름과 물이 섞여 있을 수 있나?
59
인간이 2세를 생산할 때 DNA의 결합 과정에 대해 설명해줘.
25
금성에서 달이 어떻게 보일까?
5
차량의 연비가 좋아질 때 나타나는 긍정적인 효과는?
104
혼합물의 특성에 대해 알려줘.
276
요새 너무 힘들다.
14
세제의 거품이 만들어지는 원리는?
270
나무가 생태계에서 하는 역할에 대해 설명해줘.
238
전류의 흐름을 극대화 하려면 배터리와 저항을 어떻게 연결해야 하나?
269
식물이 높이 자랄 수 있게 하는 메커니즘이 궁금해.
43
달을 보면 항상 같은 면만 보이는 이유가 뭐야?
65
식초와 베이킹 소다를 섞어주면 어떤 일이 일어나?
97
세균의 순기능에 대해 알려줄래?
206
오토마톤의 특징에 대해 알려줘.
21
다양한 책을 catalog화 하는 코드에서 class 정의 방법 알려줘.
221
전구가 병렬로 연결될 때 전류가 줄어드는 원인은?
71
물속에서 침전이 발생하는 원리에 대해 알려줘.
254
화산 폭발이 발생한 후 새로운 생물 군집이 생겨나는 것을 무엇이